In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve
from answer_judge.vqaEval import VQAEval

In [2]:
data_reader=ReadData('/root/autodl-tmp/hallscope/VizWiz/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,answer_type,answers,most_likely
0,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,"[Given the question 'What is on this page?', i...","[other, other, unanswerable, unanswerable, num...","[{'answer_confidence': ['maybe', 'yes', 'yes',...",{'embedding': [[[-0.00073624 0.0111084 -0.01...
1,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,"[Given the question 'What is this, please?', i...","[other, other, other, other, unanswerable, una...","[{'answer_confidence': ['maybe', 'yes', 'yes',...",{'embedding': [[[-0.00231934 0.02856445 0.00...
2,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,[Given the question 'Is this unsweetened or sw...,"[unanswerable, other, other, other, other, oth...","[{'answer_confidence': ['maybe', 'yes', 'yes',...",{'embedding': [[[-0.00073624 0.0111084 -0.01...
3,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,"[Given the question 'What does the say?', is...","[other, unanswerable, other, other, other, oth...","[{'answer_confidence': ['yes', 'no', 'yes', 'y...",{'embedding': [[[-0.00073624 0.0111084 -0.01...
4,[/root/autodl-fs/vizwiz/train/train/VizWiz_tra...,"[Given the question 'what spice is this?', is ...","[other, other, unanswerable, unanswerable, oth...","[{'answer_confidence': ['yes', 'yes', 'no', 'y...",{'embedding': [[[-0.00073624 0.0111084 -0.01...


In [5]:
df.shape

(1250, 5)

In [6]:
i=1

In [7]:
df.iloc[i]['img_path']

['/root/autodl-fs/vizwiz/train/train/VizWiz_train_00001984.jpg',
 '/root/autodl-fs/vizwiz/train/train/VizWiz_train_00001985.jpg',
 '/root/autodl-fs/vizwiz/train/train/VizWiz_train_00001986.jpg',
 '/root/autodl-fs/vizwiz/train/train/VizWiz_train_00001987.jpg',
 '/root/autodl-fs/vizwiz/train/train/VizWiz_train_00001988.jpg',
 '/root/autodl-fs/vizwiz/train/train/VizWiz_train_00001989.jpg',
 '/root/autodl-fs/vizwiz/train/train/VizWiz_train_00001990.jpg',
 '/root/autodl-fs/vizwiz/train/train/VizWiz_train_00001991.jpg']

In [8]:
df.iloc[i]['most_likely']['response']

['Answerable',
 'Unanswerable',
 'Unanswerable',
 'Answerable',
 'Unanswerable',
 'Unanswerable',
 'Unanswerable',
 'Unanswerable']

In [9]:
df.iloc[i]['question']

["Given the question 'What is this, please?', is the question answerable or unanswerable based on the image?\nPlease reply with 'Unanswerable' or 'Answerable'.",
 "Given the question 'What are the directions for someone who is ten years old?', is the question answerable or unanswerable based on the image?\nPlease reply with 'Unanswerable' or 'Answerable'.",
 "Given the question 'What color is this?', is the question answerable or unanswerable based on the image?\nPlease reply with 'Unanswerable' or 'Answerable'.",
 "Given the question 'What is this item.', is the question answerable or unanswerable based on the image?\nPlease reply with 'Unanswerable' or 'Answerable'.",
 "Given the question 'What is this, please?', is the question answerable or unanswerable based on the image?\nPlease reply with 'Unanswerable' or 'Answerable'.",
 "Given the question 'what is the title of this book?', is the question answerable or unanswerable based on the image?\nPlease reply with 'Unanswerable' or 'An

In [10]:
df.iloc[i]['answer_type']

['other',
 'other',
 'other',
 'other',
 'unanswerable',
 'unanswerable',
 'other',
 'unanswerable']

In [12]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [13]:
embeddings=np.concatenate(embeddings,axis=0)

In [14]:
embeddings.shape

(10000, 33, 4096)

In [15]:
gt_label=df['answer_type'].to_list()
gt_label=np.concatenate(gt_label,axis=0).tolist()
len(gt_label)

10000

In [16]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [17]:
len(rsps)

10000

In [18]:
is_correct=[]
for i,x in enumerate(gt_label):
    if gt_label[i]=='unanswerable' and rsps[i].lower()=='unanswerable':
        is_correct.append(1)
    elif gt_label[i]!='unanswerable' and rsps[i].lower()!='unanswerable':
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [19]:
is_correct

array([0, 0, 1, ..., 0, 1, 1])

In [ ]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 10000 responses. 4043 answer correctly and 5957 answer wrongly


In [ ]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 4000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:4000
validset length:4000
testset length:2000
trainset postive: 1588 negative: 2412
valset postive: 1638 negative: 2362
testset postive: 817 negative: 1183


In [37]:
all_embeddings = embeddings
all_embeddings.shape

(10000, 33, 4096)

In [38]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [39]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [50]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    split=discriminator.get_best_split(y)
    y_test=(scores>split)
    return accuracy_score(y,y_test)
    #return auroc(scores,y)

In [51]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5905
found best n_components:1,layer:2 Score: 0.60275
found best n_components:1,layer:12 Score: 0.733
found best n_components:1,layer:19 Score: 0.7405


  9%|▉         | 1/11 [00:07<01:17,  7.79s/it]

found best n_components:2,layer:19 Score: 0.75225


100%|██████████| 11/11 [01:22<00:00,  7.54s/it]


In [52]:
best_paras

{'n_components': 2, 'layer': 19, 'best_score': 0.75225}

In [53]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [54]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(hall_label_val)

In [55]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.74825 AUROC:0.7846005593359817


In [56]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.5915 AUROC:0.7585242175205456


Train Linear Prob to Get Better Results

In [57]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [58]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [59]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.7325 AUROC:0.7700967707558424
